In [11]:
import pandas as pd

# read the csv and attempt to parse column types
dev_2017_pre = pd.read_csv('consumption_deviation_FR_new.csv', sep=';', decimal=',', parse_dates=['timestamp'])

print(dev_2017_pre)

# since the quantity column is parsed generically as object, 
# it likely includes some anomalous (e.g. missing) values 
print(dev_2017_pre.dtypes)

                timestamp      quantity
0     2017-01-01 00:00:00       -44,875
1     2017-01-01 00:30:00       -18,125
2     2017-01-01 01:00:00       -10,375
3     2017-01-01 01:30:00         7,125
4     2017-01-01 02:00:00       -43,375
...                   ...           ...
17535 2017-12-31 22:45:00  -21,48982889
17536 2017-12-31 23:00:00  -7,807809406
17537 2017-12-31 23:15:00  -56,69726053
17538 2017-12-31 23:30:00   21,83256871
17539 2017-12-31 23:45:00   4,452218702

[17540 rows x 2 columns]
timestamp    datetime64[ns]
quantity             object
dtype: object


In [12]:
# rows indices 13594, 13595, 13596 are missing values for deviation
print(dev_2017_pre[dev_2017_pre['quantity']=='na'])


                timestamp quantity
13594 2017-11-10 16:00:00       na
13595 2017-11-10 16:30:00       na
13596 2017-11-10 17:00:00       na


In [15]:
# since only few rows contain missing quantity values, 
# we just load the csv again skipping them
dev_2017 = pd.read_csv('consumption_deviation_FR_new.csv', sep=';', decimal=',', parse_dates=['timestamp'], skiprows=range(13595,13598))

# check that rows with missing quantity values have been removed
print(dev_2017[dev_2017['quantity']=='na'])

Empty DataFrame
Columns: [timestamp, quantity]
Index: []


In [16]:
# the column types are correctly parsed
dev_2017.dtypes

timestamp    datetime64[ns]
quantity            float64
dtype: object